# VacationPy
A map view of some weather conditions around the world with names and locations of hotels in cities with ideal weather conditions.<br><br>
Code by: Ricardo G. Mora, Jr.  (10/22/2021)<br><br>
City names provided by: CityPy (https://pypi.org/project/citipy)<br>
Weather data proved by: OpenWeather (https://openweathermap.org)<br>
Hotel names provided by: Google Places (https://developers.google.com/maps/)<br><br>
#### Note:
* To run this notebook's code, you will need an API key from Google Maps.  Name it "g_key" and place it in a file named "api_keys" in this notebook's directory.<br>
* WeatherPy must be run in its entirety prior to running this notebook in order to generate the csv file containing the list of cities needed.  The csv file needed is "cities.csv" and it should be found in folder "output_data".

In [1]:
# Dependencies
import pandas as pd
import requests
import gmaps
from pprint import pprint

# Hide warnings during execution
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import g_key

# Input File (CSV)
try:
    input_data_file = "output_data/cities.csv"
except:
    print("The input file and/or it's containing folder is missing.")

### Store WeatherPy results into DataFrame
* Load the csv file created by WeatherPy into a DataFrame.

In [2]:
# Load weather data csv file into dataframe
world_weather_data = pd.read_csv(input_data_file, index_col=0)
world_weather_data.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date (Unix)
0,castro,BR,-24.7911,-50.0119,72.86,57,100,6.78,1635017182
1,sur,OM,22.5667,59.5289,81.70,60,0,7.65,1635017341
2,torbay,CA,47.6666,-52.7314,63.37,90,75,13.80,1635017342
3,jamestown,US,42.0970,-79.2353,51.39,92,90,5.01,1635017259
4,faya,SA,18.3851,42.4509,63.12,28,3,6.02,1635017344


### Humidity Heatmap
* Configure gmaps.
* Use the latitude and longitude as locations and humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create list of latitude/longitude tuples for gmaps
coordinateslist = world_weather_data[["Latitude", "Longitude"]]

# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Set up the map layout
figure_layout = {
    "width": "800px",
    "height": "600px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}

# Set up the heatmap layer for humidity
humidity_layer = gmaps.heatmap_layer(
    coordinateslist, 
    weights=world_weather_data["Humidity (%)"], 
    dissipating=False, 
    max_intensity=100, 
    point_radius=2
)

# Display map
fig = gmaps.figure(layout=figure_layout)                                    
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Define search parameters
idealmaxtemp = 80
idealmintemp = 70
idealmaxwindspeed = 10
idealmaxcloudiness = 25
idealmaxhumidity = 75

# Filter dataframe to contain only cities with ideal conditions
ideal_cities_data = world_weather_data.loc[
    (world_weather_data["Max Temp (F)"] <= idealmaxtemp) &
    (world_weather_data["Max Temp (F)"] >= idealmintemp) &
    (world_weather_data["Wind Speed (mph)"] <= idealmaxwindspeed) &
    (world_weather_data["Cloudiness (%)"] <= idealmaxcloudiness) &
    (world_weather_data["Humidity (%)"] <= idealmaxhumidity)
]

# Drop any rows with null values and re-number the indices
ideal_cities_data.dropna()
ideal_cities_data.reset_index(drop=True, inplace=True)

# Display dataframe
ideal_cities_data.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date (Unix)
0,saint george,US,37.1041,-113.5841,77.47,22,1,7.00,1635017337
1,salalah,OM,17.0151,54.0924,77.09,73,0,3.44,1635017339
2,ponta do sol,PT,32.6667,-17.1000,70.43,54,25,6.51,1635017416
3,diu,IN,20.7141,70.9822,76.42,68,0,8.99,1635017432
4,russell,US,32.3502,-85.1999,76.28,40,1,0.00,1635017526


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_cities_data
hotel_df.insert(9, "Hotel Name", "")
hotel_df.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date (Unix),Hotel Name
0,saint george,US,37.1041,-113.5841,77.47,22,1,7.00,1635017337,
1,salalah,OM,17.0151,54.0924,77.09,73,0,3.44,1635017339,
2,ponta do sol,PT,32.6667,-17.1000,70.43,54,25,6.51,1635017416,
3,diu,IN,20.7141,70.9822,76.42,68,0,8.99,1635017432,
4,russell,US,32.3502,-85.1999,76.28,40,1,0.00,1635017526,


#### Note:  This block of code takes a few minutes to execute.

In [7]:
# Set the google maps query string
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set the non-changing parameters
params = {
    "keyword": "hotel",
    "rankBy": "distance",
    "type": "lodging",
    "radius": 5000,
    "key": g_key
}

# Loop through dataframe and get nearest hotel name for each city using API calls to Google Places
for index, row in hotel_df.iterrows():
    
    # Set the location parameter and call google places
    params["location"] = f"{row['Latitude']}, {row['Longitude']}"
    response = requests.get(base_url, params=params).json()

    # If a hotel was found, save it's name in the dataframe, otherwise specify "None"
    try:
        hotel_df.loc[index, ["Hotel Name"]] = response["results"][0]["name"]
    except:
        hotel_df.loc[index, ["Hotel Name"]] = "None"

# Display dataframe
hotel_df.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date (Unix),Hotel Name
0,saint george,US,37.1041,-113.5841,77.47,22,1,7.00,1635017337,Red Lion Hotel & Conference Center St. George
1,salalah,OM,17.0151,54.0924,77.09,73,0,3.44,1635017339,Salalah Gardens Hotel
2,ponta do sol,PT,32.6667,-17.1000,70.43,54,25,6.51,1635017416,Hotel do Campo
3,diu,IN,20.7141,70.9822,76.42,68,0,8.99,1635017432,Hotel TGH - The Grand Highness
4,russell,US,32.3502,-85.1999,76.28,40,1,0.00,1635017526,None


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=hotel_info
)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…